# Parte 2 - Laboratorio 4
### Sara Zavala
### Ricardo Valenzuela

## Dataset

In [198]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import *
from sklearn import metrics
from quickda.explore_data import *

In [199]:
df_init = pd.read_csv("VirusSample.csv")

In [200]:
df_init.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus


## Analisis exploratorio preliminar

In [201]:
explore(df_init, method="summarize")

,dtypes,count,null_sum,null_pct,nunique,min,25%,50%,75%,max,mean,median,std,skew
api,object,9795,0,0.0,2980,?changeGuard@QMetaObject@@SAXPAPAVQObject@@PAV...,-,-,-,"wvsprintfW,VarDecFix,GetUserDefaultLangID,GetC...",-,-,-,-
class,object,9795,0,0.0,13,Adware,-,-,-,Worms,-,-,-,-
file,object,9795,0,0.0,9795,0003103d2781125e0fd463e5042d3f26cb932f01,-,-,-,fffa9307c95121145fb056748bc830d1724f81d1,-,-,-,-


In [202]:
df_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9795 entries, 0 to 9794
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   file    9795 non-null   object
 1   api     9795 non-null   object
 2   class   9795 non-null   object
dtypes: object(3)
memory usage: 229.7+ KB


Solo tenemos 3 columnas iniciales, por lo que de momento no es necesario hacer un pandas profiling.

In [203]:
df_init["class"].value_counts()

Trojan        6153
Virus         2367
Backdoor       447
Worms          441
Adware         222
Agent          102
Downloader      31
Spyware         11
Ransomware      10
Riskware         4
Dropper          4
Crypt            2
Keylogger        1
Name: class, dtype: int64

Tenemos 13 diferentes clases, sin embargo, vemos tambien que el dataset esta muy desbalanceado, la diferencia entre su clase con mas incidencias con la de menos es demasiado grande. Decidimos quitar aquellas clases que tienen menos de 100 incidencias, esto ya que no queremos agregar overfitting al intentar balancear demasiado las clases que tienen pocos registros. Tambien tomamos esta decision ya que se querian dejar solamente las clases que tiene el articulo de referencia, esto con el fin de hacer una mejor comparacion.

In [204]:
df_init.drop(df_init[df_init["class"] == "Downloader"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Ransomware"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Spyware"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Riskware"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Dropper"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Crypt"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Keylogger"].index, inplace=True)

In [205]:
df_init["class"].value_counts()

Trojan      6153
Virus       2367
Backdoor     447
Worms        441
Adware       222
Agent        102
Name: class, dtype: int64

Transformamos la clase a una variable cuantitativa para facilidad al crear modelos:  
    - 0: Adware  
    - 1: Agent  
    - 2: Backdoor  
    - 3: Trojan  
    - 4: Virus  
    - 5: Worms  

In [206]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_init['class'] = le.fit_transform(df_init['class'])

In [207]:
df_init["class"].value_counts()

3    6153
4    2367
2     447
5     441
0     222
1     102
Name: class, dtype: int64

Obtenemos conteo de palabras:

In [208]:
features = df_init.loc[:, df_init.columns != 'class']
target = df_init["class"]

In [209]:
features_df = pd.DataFrame(features,dtype=str)
features_df.apply(le.fit_transform)

features_df = pd.DataFrame(features_df.apply(le.fit_transform).values[:,:])
features_df

,0,1
0,4887,1240
1,3104,599
2,9414,2501
3,2027,2245
4,83,2127
...,...,...
9727,2242,1611
9728,3233,2059
9729,8390,2708
9730,7864,1209


Balancemos un poco los datos, siempre manteniendo el patron para no agregar overfitting

In [210]:
from collections import Counter
print("Tamanio completo del df tras preprocesamiento: ", len(df_init))
print("Antes del oversampling: ",Counter(target))

Tamanio completo del df tras preprocesamiento:  9732
Antes del oversampling:  Counter({3: 6153, 4: 2367, 2: 447, 5: 441, 0: 222, 1: 102})


In [211]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = {0: 1750, 1:1500, 2:2500,4:3000, 5:2250})

x, y = sm.fit_resample(features_df, target)

oversampled_data = x

print("Luego del oversampling: ",Counter(y))

Luego del oversampling:  Counter({3: 6153, 4: 3000, 2: 2500, 5: 2250, 0: 1750, 1: 1500})


Ahora escalamos los datos con el fin de no ver tanta diferencia entre datos con las estadisticas y graficas

In [212]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
features_df = pd.DataFrame(scaler.transform(x))
features_df

,0,1
0,0.502209,0.421912
1,0.318981,0.203811
2,0.967424,0.850970
3,0.208303,0.763865
4,0.008529,0.723716
...,...,...
17148,0.286507,0.498809
17149,0.592745,0.229330
17150,0.930737,0.498809
17151,0.063611,0.798571


Finalmente dividimos los datos

In [213]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features_df, y, test_size=0.3, random_state=42)

## KNN

In [214]:
from sklearn.neighbors import KNeighborsClassifier
#Creacion de modelo
knn = KNeighborsClassifier()
knn.fit(features_train, target_train)

KNeighborsClassifier()

In [215]:
from sklearn import metrics
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_train = knn.predict(features_train)
print("El valor de accuracy con la data de entrenamiento es de: ", metrics.accuracy_score(predict_train, target_train))

El valor de accuracy con la data de entrenamiento es de:  0.7976180561339219


In [216]:
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_test = knn.predict(features_test)
print("El valor de accuracy con la data de test es de: ", metrics.accuracy_score(predict_test, target_test))

El valor de accuracy con la data de test es de:  0.7201710066070734


In [217]:
from sklearn.metrics import *
#Matriz de confusion
print(classification_report(target_test, predict_test))

              precision    recall  f1-score   support

           0       0.62      0.63      0.62       516
           1       0.72      0.82      0.77       437
           2       0.87      0.94      0.91       756
           3       0.73      0.78      0.76      1875
           4       0.66      0.52      0.58       918
           5       0.64      0.56      0.60       644

    accuracy                           0.72      5146
   macro avg       0.71      0.71      0.71      5146
weighted avg       0.71      0.72      0.71      5146



In [218]:
cf = confusion_matrix(target_test, predict_test)
cf

array([[ 325,   16,    8,   86,   59,   22],
       [   8,  359,    4,   39,   22,    5],
       [   4,    1,  713,   32,    4,    2],
       [  73,   76,   38, 1469,  114,  105],
       [  94,   34,   44,  199,  477,   70],
       [  24,   14,   10,  183,   50,  363]], dtype=int64)

In [219]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
cross_val_score(knn, features_train, target_train, scoring='accuracy', cv=cv, n_jobs=-1)

array([0.71690258, 0.72522898, 0.69775187, 0.70274771, 0.69941715,
       0.7194005 , 0.71440466, 0.71166667, 0.69166667, 0.70416667])

Modelo considerablemente bueno, vemos que sus mejores clasificaciones ocurren con Agent, Backdoor y Trojan, sin embargo, para las demas clases su valor de precicion no estan bueno.

# Classification Tree

In [220]:
from sklearn import tree
clfTree = tree.DecisionTreeClassifier()
clfTree.fit(features_train,target_train)

DecisionTreeClassifier()

In [221]:
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_train = clfTree.predict(features_train)
print("El valor de accuracy con la data de entrenamiento es de: ", metrics.accuracy_score(predict_train, target_train))

El valor de accuracy con la data de entrenamiento es de:  0.9987507287415675


In [222]:
predict_test = clfTree.predict(features_test)
print("El valor de accuracy con la data de test es de: ", metrics.accuracy_score(predict_test, target_test))

El valor de accuracy con la data de test es de:  0.7848814613291877


In [223]:
from sklearn.metrics import *
#Matriz de confusion
print(classification_report(target_test, predict_test))

              precision    recall  f1-score   support

           0       0.66      0.67      0.67       516
           1       0.74      0.76      0.75       437
           2       0.93      0.93      0.93       756
           3       0.86      0.84      0.85      1875
           4       0.69      0.69      0.69       918
           5       0.66      0.68      0.67       644

    accuracy                           0.78      5146
   macro avg       0.76      0.76      0.76      5146
weighted avg       0.79      0.78      0.79      5146



In [224]:
cf = confusion_matrix(target_test, predict_test)
cf

array([[ 348,    7,    4,   27,   96,   34],
       [  13,  334,    1,   44,   33,   12],
       [   4,    2,  704,   29,   14,    3],
       [  42,   57,   26, 1582,   72,   96],
       [  91,   35,   18,   64,  635,   75],
       [  27,   14,    5,   85,   77,  436]], dtype=int64)

In [225]:
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
cross_val_score(clfTree, features_train, target_train, scoring='accuracy', cv=cv, n_jobs=-1)

array([0.78517902, 0.76852623, 0.77601998, 0.77268943, 0.77685262,
       0.7868443 , 0.77019151, 0.75666667, 0.77333333, 0.7775    ])

Nuestro mejor modelo, vemos que tiene una media de acurracy de 0.78 el cual es de nuestros mejores resultados, la mejor clase que categorizo correctamiente fue la de Backdoor seguida por trojan, y la de peor clasificacion fue Advware. Sin embargo, vemos una gran variedad de precision al comparar todas las clases, vemos que la mayoria de confusiones suceden con la clase virus.

# Naive Bayes

In [226]:
from sklearn.naive_bayes import MultinomialNB
#Creacion de modelo
naiveBB = MultinomialNB()
naiveBB.fit(features_train, target_train)

MultinomialNB()

In [227]:
from sklearn import metrics
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_train = naiveBB.predict(features_train)
print("El valor de accuracy con la data de entrenamiento es de: ", metrics.accuracy_score(predict_train, target_train))

El valor de accuracy con la data de entrenamiento es de:  0.3562921629049721


In [228]:
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_test = naiveBB.predict(features_test)
print("El valor de accuracy con la data de test es de: ", metrics.accuracy_score(predict_test, target_test))

El valor de accuracy con la data de test es de:  0.36436066848037313


In [229]:
#Matriz de confusion
print(classification_report(target_test, predict_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       516
           1       0.00      0.00      0.00       437
           2       0.00      0.00      0.00       756
           3       0.36      1.00      0.53      1875
           4       0.00      0.00      0.00       918
           5       0.00      0.00      0.00       644

    accuracy                           0.36      5146
   macro avg       0.06      0.17      0.09      5146
weighted avg       0.13      0.36      0.19      5146



C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [230]:
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
cross_val_score(naiveBB, features_train, target_train, scoring='accuracy', cv=cv, n_jobs=-1)

array([0.3438801 , 0.36136553, 0.33555371, 0.367194  , 0.34471274,
       0.34471274, 0.35470441, 0.38666667, 0.37083333, 0.35333333])

Nuestro peor modelo con referencia, este modelo tomo como unica clasificacion valida los ataques de tipo Trojan y clasifico a todas las observaciones como la misma, esto causa que nuestro accuracy sea solamente de 0.333, la metrica de recall para esta catergoria es de 1 pero el f1 y precision son muy bajos. Esto sucede ya que el modelo se sobre ajusto al dataset y tomo la via facil de clasificar como un solo tipo a todos los registros, este modelo no se tomara para la comparacion con los resultados del articulo.

# ¿se lograron obtener mejores métricas que las obtenidas en el artículo para la clasificación de malware?

A simple vista podemos ver que en su mayoria todas las clases de nuestro mejor modelo tuvieron un peor resultado que los del articulo, solo el caso de Troyan fue el que tuvo mejores resultados. Esto se puede deber a una variedad de factores, sin embargo, el principal problema que encontramos al trabajar con la RAW data fue que estaba muy desbalanceada, la diferencia entre la clase que tenia mas incidencias y la de menor era demasiada, nosotros intentamos balancearla con overfitting pero esto pudo causar que el modelo se sobre ajustara a los datos (como en el caso de bayes ingenuo). A pesar de que los datos con los que se trabajo estaban bastante completos para unos casos no fue suficiente. Sin emabargo, creemos que apesar te no lograr los mejores resultados nos acercamos bastante apesar de contar con las complicaciones antes mencionadas. Tambien creemos que aplicando otras tecnicas de procesamiento de datos hubiesemos obtenido aun mejores resultados, se planteo usar algun bag of words, pero esto no iba con los objetivos del laboratorio ya que el principal objetivo del lab era clasificar los ataques con base a la secuencia de APIs que se llaman no solo las APIs en especifico.

- 0: Adware  
- 1: Agent  
- 2: Backdoor  
- 3: Trojan  
- 4: Virus  
- 5: Worms  

Nuestro mejor modelo (clasification tree):  

 	 	precision    recall  f1-score   support

           0       0.66      0.67      0.67       516
           1       0.74      0.76      0.75       437
           2       0.93      0.93      0.93       756
           3       0.86      0.84      0.85      1875
           4       0.69      0.69      0.69       918
           5       0.66      0.68      0.67       644

Resultados articulo:

					HGB 	RF  	SVM  	XGB  
		Malware Types 		F1-score F1-Score F1-Score F1-Score  
		Adware 			0.89 	0.92 	0.96 	0.91  
		Agent 			0.91 	0.91 	0.91 	0.91  
		Backdoor 		0.95 	0.95 	0.94 	0.95  
		Trojan 			0.75 	0.72 	0.76 	0.76  
		Virus 			0.95 	0.79 	0.97 	0.98  
		Worms 			0.92 	0.68 	0.92 	0.90  